# AI FAQ 챗봇 만들기 with RAG

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.schema import Document
import json

1. 데이터 로드

In [3]:
with open('./data/faq_chatbot_data.json', 'r', encoding='utf-8') as f:
    faq_data = json.load(f)

In [4]:
faq_data

[{'question': '반품 정책이 어떻게 되나요?',
  'answer': '제품을 수령한 후 14일 이내에 반품이 가능합니다. 반품 신청은 고객센터에서 도와드립니다.'},
 {'question': '배송은 얼마나 걸리나요?',
  'answer': '일반적으로 2~5일 정도 소요되며, 지역에 따라 다를 수 있습니다.'},
 {'question': '회원가입 없이 주문할 수 있나요?',
  'answer': '네, 비회원 주문이 가능합니다. 다만, 주문 조회 및 혜택을 받기 위해 회원가입을 추천드립니다.'},
 {'question': '포인트 적립은 어떻게 되나요?',
  'answer': '구매 금액의 1%가 적립되며, 특별 프로모션 기간 동안 추가 적립이 가능합니다.'},
 {'question': '해외 배송이 가능한가요?',
  'answer': '일부 국가를 대상으로 해외 배송이 가능하며, 배송비는 지역에 따라 상이할 수 있습니다.'},
 {'question': '제품 보증 기간은 얼마나 되나요?',
  'answer': '제품 보증 기간은 기본적으로 1년이며, 일부 제품은 연장 보증이 가능합니다.'},
 {'question': 'A/S 신청은 어떻게 하나요?',
  'answer': '공식 홈페이지 또는 고객센터를 통해 A/S 신청이 가능합니다.'},
 {'question': '재고가 없는 상품은 언제 다시 입고되나요?',
  'answer': '입고 일정은 상품마다 다르며, 알림 설정을 통해 입고 시 안내받을 수 있습니다.'},
 {'question': '구매 후 영수증을 받을 수 있나요?',
  'answer': '네, 주문 내역에서 전자 영수증을 다운로드할 수 있습니다.'},
 {'question': '다른 사람이 대신 주문을 수령할 수 있나요?',
  'answer': '네, 수령인을 지정할 수 있으며 신분증 확인 후 수령 가능합니다.'},
 {'question': '결제 방법에는 어떤 것이 있나요?',
  'answer': '신용카드, 

2. embedding 모델 생성 + vector store 생성

In [5]:
embedding_model = OpenAIEmbeddings()

vector_store = Chroma(
    embedding_function=embedding_model,
    persist_directory='./chroma_db'
)

documents = [Document(page_content=item["question"]+" "+item["answer"]) for item in faq_data]
vector_store.add_documents(documents)

C:\Users\USER\AppData\Local\Temp\ipykernel_18132\2870071854.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


['70abaa62-ed25-4dee-9fa8-6d0273abb9f8',
 '3f07e149-4559-4094-81aa-c211a98ea1d2',
 '247cba95-e823-4f26-aae1-bfb6c4a3f83f',
 '09df8d72-eeab-4bc0-848a-733c41276162',
 'c6e72f85-9f3c-436f-8f6c-dc903d31de4d',
 'c70a427b-93fd-43ca-b170-b8c5102071a5',
 '0d1c491f-6147-47a1-a6ac-099b3917cb0f',
 'cd828428-0b05-436c-a64a-3554ef630b32',
 'af13b0ba-3e37-48e9-be7a-0570d7f0359b',
 '65e5c0fc-dfb1-4dd9-8a28-be60955264ae',
 '1e7daff4-a9f0-4ba6-b0c1-acd686a0f894',
 '5027440a-a498-4c04-bcaf-54a4175b3bdc',
 '81e2274d-395d-4173-b2d4-77202545ca51']

3. llm 설정 (모델 생성)

In [6]:
model = ChatOpenAI(model='gpt-4o-mini', temperature=0.5)

4. 프롬프트 및 체인 설정

In [ ]:
prompt = PromptTemplate(
    template="""
다음 '질문'에 대해 '문서'의 내용만을 참고하여 '답변'을 생성해 주세요.
질문 : {question}
문서 : {context}
답변 : 
""",
    input_variables=['question', 'context']
)

# prompt를 구성할 때 retriever의 검색 결과로 들어갈 input_variable은 'context'로 설정해야 함

In [8]:
retriever = vector_store.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001F1F068E970>, search_kwargs={})

In [9]:
qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt}
)

5. 사용

In [10]:
qa_chain.run('배송 얼마나 걸려?')

C:\Users\USER\AppData\Local\Temp\ipykernel_18132\834878008.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa_chain.run('배송 얼마나 걸려?')


'배송은 일반적으로 2~5일 정도 소요되며, 지역에 따라 다를 수 있습니다.'

In [11]:
qa_chain.run('구매 취소하고 싶은데')

'상품이 출고되지 않은 상태라면 주문 취소가 가능합니다.'

In [12]:
qa_chain.run('구매 영수증 좀 줘')

'구매 후 영수증은 주문 내역에서 전자 영수증을 다운로드하여 받을 수 있습니다.'